In [1]:
import requests
from bs4 import BeautifulSoup as soup
import re

In [2]:
index_soup = soup(requests.get('https://www.bild.de/').text)

In [3]:
def get_links_of_date(day, month, year):
    
    url = f"https://www.bild.de/archive/{year}/{month}/{day}/index.html"
    
    # Search function to only find p tags containing extactly one 'a' tag having a href attr 
    def search_function(tag, filter_href= lambda x: True):
        if not tag.name == 'p':
            return False
        containing_a = tag.find_all('a')
        if not len(containing_a) == 1:
            return False
        a = containing_a[0]
        if not 'href' in a.attrs:
            return False
        if 'bild-plus' in a['href']:            # filter bild-plus
            return False
        if 'video/clip' in a['href']:           # filter videos
            return False 
        if not filter_href(a['href']): # only articles in politic
            return False
        return True

    # Filter href for keywords
    def filter_href(href):
        if not href.startswith('/politik') and not href.startswith('/news'):
            return False
        if not 'ukraine' in href:
            return False
        return True

    
    
    html_text = requests.get(url).text

    archiv_soup = soup(html_text)
    
    list_articles = archiv_soup.find('div', class_='txt')
    
    ex = list_articles.find_all(lambda x: search_function(x, filter_href))

    return [{'href':"https://www.bild.de"+p.find('a')['href'], 'title': p.find('a').text} for  p in ex]

In [4]:
get_links_of_date(22,4,2022)

[{'href': 'https://www.bild.de/politik/inland/politik-inland/ukraine-chancenlos-ueber-diese-deutschen-experten-freut-sich-putin-79854620.bild.html',
  'title': 'Ukraine chancenlos? Über diese deutschen Experten freut sich Putin'},
 {'href': 'https://www.bild.de/politik/ausland/politik-ausland/ukraine-krieg-dj-zoff-wegen-putins-krieg-plattenfirmen-muessen-vertraege-kuendig-79853820.bild.html',
  'title': 'Ukraine-Krieg - DJ-Zoff wegen Putins Krieg: „Plattenfirmen müssen Verträge kündigen!“'},
 {'href': 'https://www.bild.de/politik/ausland/politik/ukraine-briten-zeitung-fordert-sanktionen-gegen-deutschland-wegen-putins-gas-79838242.bild.html',
  'title': 'Ukraine: Briten-Zeitung fordert Sanktionen gegen Deutschland – wegen Putins Gas'},
 {'href': 'https://www.bild.de/politik/ausland/politik-ausland/ukraine-krieg-gegen-putins-truppen-soll-phoenix-ghost-drohne-helfen-79843232.bild.html',
  'title': 'Ukraine-Krieg: Gegen Putins Truppen soll „Phoenix Ghost“-Drohne helfen'},
 {'href': 'https:

In [5]:
url = 'https://www.bild.de/politik/ausland/politik-ausland/krieg-in-der-ukraine-putin-will-negative-szenarien-verhindern-79006554.bild.html'

In [6]:
url2 = 'https://www.bild.de/politik/2022/politik/krieg-in-der-ukraine-neues-massengrab-mit-9000-leichen-entdeckt-79845084.bild.html'

In [7]:
def get_article_content(url):
    article_soup = soup(requests.get(url).text)
    title = article_soup.find('span', class_='headline')
    title_txt = title.text

    kicker = article_soup.find('span', class_='kicker')
    kicker_txt = kicker.text

    authors = article_soup.find('span', class_='authors__name')
    authors_txt = authors.text

    figcaptions = article_soup.find_all('figcaption')
    figcaption_txt = ' '.join([a.contents[0] for a in figcaptions])

    text_paragraphs = article_soup.find('div', class_='txt').find_all('p', recursive=False)
    txt = ' '.join([a.text for a in text_paragraphs])
    
    pubdate = article_soup.find('time', class_='authors__pubdate').text
    
    dictionary = {
        'title': title_txt,
        'kicker': kicker_txt,
        'authors': authors_txt,
        'figcaptions': figcaption_txt,
        'text':txt,
        'date': pubdate
    }
    return dictionary

In [8]:
get_article_content(url2)

{'title': 'Verscharrte Putin hier die Kriegsopfer von Mariupol?',
 'kicker': 'Neues Massengrab entdeckt',
 'authors': 'Matthias Hoffmann',
 'figcaptions': '\n                            VORHER: Die Aufnahme stammt vom 19. März. Unterhalb der Straße sind keine Aushebungen zu sehen \n                            NACHHER: Das gleiche Gebiet, unterhalb der Straße das mutmaßliche Massengrab (Aufnahme vom 3.\xa0April) \n',
 'text': 'Die russischen Gräueltaten nehmen kein Ende. In Manhusch in der Nähe von Mariupol soll ein Massengrab mit bis zu 9000 Toten entdeckt worden sein. Das schreibt der Bürgermeister der inzwischen nahezu vollständig zerstörten Stadt bei Telegram. Wadym Bojtschenko wirft den Russen vor, „ihre Kriegsverbrechen verheimlichen“ zu wollen. Sie hätten die Leichen von Zivilisten aus Mariupol geschafft und in dem rund 20 Kilometer westlich gelegenen Städtchen verscharrt. Seit dem 1. März belagern Putins Truppen die Stadt, haben sie in Grund und Boden gebombt. Tausenden Zivilist